# Objects & classes

## Redefinition of a class member function

In the case of simple public inheritance, it is possible to define new member functions in the derived class, as well as to override those defined in the base class:

In [1]:
#include<iostream>

In [2]:
struct Base
 {
  void function1() { std::cout<<"Base::function1"<<std::endl ; }
  void function2() { std::cout<<"Base::function2"<<std::endl ; }
 } ;

In [3]:
struct Derived : public Base
 {
  void function2() { std::cout<<"Derived::function2"<<std::endl ; }
  void function3() { std::cout<<"Derived::function3"<<std::endl ; }
 } ;

In [4]:
Derived obj ;
obj.function1() ;
obj.function2() ;
obj.function3() ;

Base::function1
Derived::function2
Derived::function3


## Overloading and hiding

Now let's discuss function overloading, i.e., using multiple functions with the same name but different signatures (the number or type of arguments). How does it work?

In [5]:
struct Base
 {
  void display( int ) { std::cout<<"Base::display( int )"<<std::endl ; }
  void display( float ) { std::cout<<"Base::display( float )"<<std::endl ; }
 } ;

In [6]:
struct Derived : public Base
 {
  void display( float ) { std::cout<<"Derived::display( float )"<<std::endl ; }
  void display( double ) { std::cout<<"Derived::display( double )"<<std::endl ; }
 } ;

In [7]:
void check( Derived obj )
 { 
  obj.display(42) ;
  obj.display(3.14f) ;
  obj.display(3.14) ;
 }

input_line_13:3:7: error: call to member function 'display' is ambiguous
  obj.display(42) ;
  ~~~~^~~~~~~
input_line_12:3:8: note: candidate function
  void display( float ) { std::cout<<"Derived::display( float "<<std::endl ; }
       ^
input_line_12:4:8: note: candidate function
  void display( double ) { std::cout<<"Derived::display( double )"<<std::endl ; }
       ^


Interpreter Error: 

When compiling `obj.mf(...)`, the compiler follow those steps:
1. search some member function named `mf` in the class of `obj`;
2. while not found at least one `mf`, move to its base classes, one after the other;
3. in the selected class, within all overloaded `mf` functions, select the one which fit better the call arguments `(...)`.

In the previous example, the compiler first select the `Derived` class, then hesitate to convert its `int` argument into either a `float` or a `double`.

### Good old-fashioned practice

When you redefine an inherited member function, **redefine all the base class functions which share the same name**, or you will hide some of them.

## Virtual member functions

Let's make all the member functions virtual, and modify `check()` so to take as input a reference to the base class, so that we can provide any derived class instance.

In [8]:
#include<iostream>

In [9]:
struct Base
 {
  virtual void display( int ) { std::cout<<"Base::display( int )"<<std::endl ; }
  virtual void display( float ) { std::cout<<"Base::display( float )"<<std::endl ; }
 } ;

In [10]:
struct Derived : public Base
 {
  virtual void display( float ) { std::cout<<"Derived::display( float "<<std::endl ; }
  virtual void display( double ) { std::cout<<"Derived::display( double )"<<std::endl ; }
 } ;

In [11]:
void check( Base * obj )
 {
  obj->display(42) ;
  obj->display(3.14f) ;
  obj->display(3.14) ;
 }

input_line_17:5:8: error: call to member function 'display' is ambiguous
  obj->display(3.14) ;
  ~~~~~^~~~~~~
input_line_15:3:16: note: candidate function
  virtual void display( int ) { std::cout<<"Base::display( int )"<<std::endl ; }
               ^
input_line_15:4:16: note: candidate function
  virtual void display( float ) { std::cout<<"Base::display( float )"<<std::endl ; }
               ^


Interpreter Error: 

The ambiguous line is now `obj.display(3.14)`: the type of `*obj` is `Base`, and the compiler hesitate to convert its `double` argument into either an `int` or a `float`.

### Good old-fashioned practice

When you define a base class with virtual functions, overload those functions for all the expected type of arguments that you expect in the derived classes.

In [12]:
struct Base
 {
  virtual void display( int ) { std::cout<<"Base::display( int )"<<std::endl ; }
  virtual void display( float ) { std::cout<<"Base::display( float )"<<std::endl ; }
  virtual void display( double ) { std::cout<<"Base::display( double )"<<std::endl ; }
 } ;

### Other good old-fashioned practice

**Make the destructor `virtual`**. Indeed, you may create a derived object with `new` and store its address in a base class pointer. When calling `delete` on this pointer, one will bypass the derived destructor, unless it is virtual.

In [ ]:
struct Base
 {
  virtual void display( int ) { std::cout<<"Base::display( int )"<<std::endl ; }
  virtual void display( float ) { std::cout<<"Base::display( float )"<<std::endl ; }
  virtual void display( double ) { std::cout<<"Base::display( double )"<<std::endl ; }
  virtual ~Base() {}
 } ;

### The cost of virtual functions

When a class has at least one virtual member function:

1. **the size of all objects**, for this class and any derived class, **is increased by the size of a pointer** (pointing to the *virtual table* of the object real class).

2. for each call to a virtual function, the compiler **inserts** in the binary **a redirection** through the object *virtual table*, so to reach the object real class function.

### The recommended use of virtual functions

Because of the cost, virtual functions are rather used for large and not numerous objects : the upper software layers of your application. For what concerns the numerous and small objects, it is preferable to use templates (which slow down the compilation but have not effect during execution).

## Function-objects

If a class supply an `operator()`, its objects can behave similarly to a function. The member variables of the class can be seen as extraneous parameters for this function.

In [13]:
#include <iostream>

In [22]:
class LinearFunction
 {
  public :
    LinearFunction( double constant ) : m_constant(constant) {}
    double operator()( double value ) { return m_constant*value ; }
  private :
    double m_constant ;    
 } ;

In [23]:
LinearFunction times2(2) ;
const int SIZE = 10 ;
double values[SIZE] = { 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 } ;
for ( int i=0 ; i<SIZE ; ++i )
 { std::cout << times2(values[i]) << " " ; }

0 2 4 6 8 10 12 14 16 18 

### The recommended use of function-objects

This type of classes is especially used with algorithms of the standard library.

Below, we review the previous function-object, in order to integrate the call to `std::cout`, and then we combine it with `std::foreach`.

In [16]:
#include <algorithm>

In [24]:
class LinearFunctionPrint

 {
  public :
    LinearFunctionPrint( double constant ) : m_constant(constant) {}
    void operator()( double value )
     { std::cout << (m_constant*value) << " " ; }
  private :
    double m_constant ;    
 } ;

In [25]:
const int SIZE = 10 ;
double values[SIZE] = { 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 } ;
std::for_each(values,values+SIZE,LinearFunctionPrint(2)) ;

0 2 4 6 8 10 12 14 16 18 

Objet-functions are classes. They can also use inheritance:

In [20]:
class LinearFunctionPrint : public LinearFunction
 {
  public :
    LinearFunctionPrint( double constant ) : LinearFunction(constant) {}
    void operator()( double value )
     { std::cout << LinearFunction::operator()(value) << " " ; }
 } ;

In [21]:
const int SIZE = 10 ;
double values[SIZE] = { 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 } ;
std::for_each(values,values+SIZE,LinearFunctionPrint(2)) ;

0 2 4 6 8 10 12 14 16 18 

### Soon...

This approach paves the way for a future parallel version of `for_each`, which will automate vectorization and/or multi-threaded execution.

## Conclusion

Modern C++ will not revolutionize the object-orientated features, but rather bring many small improvements for all the identified flaws. With the exception of function-objects, which will be mostly replaced with the incredible **lambda functions**, which brings functional programming within C++.

## Questions ?

© *CNRS 2020*  
*This document was created by David Chamont and translated by Olga Abramkina. It is available under the [Licence Creative Commons - Attribution - No commercial use - Shared under the conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*